In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

# Read in data

In [ ]:
# 读取训练数据
df = pd.read_csv ('train.csv')
df['year'] = df['year'].map(lambda x: x-2000)
#df = df.iloc[:, 1:]

# Read in test data set
df1 = pd.read_csv('test.csv')
#df1['personid'] = df1['personid'].astype(str)
#df1['year'] = df1['year'].map(lambda x: x-2000)
#df1 = df1_org.iloc[:, 1:]

# Deal with missing value

In [ ]:
import math
miss_list = []
for i in range(1205):
  miss_list.append(math.ceil(df.iloc[:,i].isnull().sum() / 1000))

miss_level, var_count = np.unique(miss_list, return_counts=True)
print(miss_level)
print(var_count)

#plt.bar(left=range(1,19), height=var_count, width=0.4, alpha=0.8, color='red')
n, bins, patches = plt.hist(x=miss_list, bins=19, color='#0504aa',
                    alpha=0.7, rwidth=0.85)


# Data Preprocessing

In [ ]:
# Impute missing values using mode
imp_freq = SimpleImputer(strategy="most_frequent")
i = 0
dele_list = []
ave_list = []
while i < 1205:
  name = 'x'+str(i+1)
  if df.iloc[:,i+3].isnull().sum() > 16000:
    dele_list.append(name)
    i+=1
    continue
  value = df.iloc[:,i+3].values.reshape(-1,1)
  imp = imp_freq.fit_transform(value)
  col = df.iloc[:,i+3].dropna()
  values, counts = np.unique(col, return_counts=True)
  freq_ind = np.argmax(counts)
  ave_list.append(values[freq_ind])
  df.iloc[:,i+3] = imp
  i += 1

print(len(ave_list))
print(len(dele_list))

962
243


In [ ]:
# Impute missing values using mean
imp_mean = SimpleImputer()
i = 0
dele_list = []
ave_list = []
while i < 1205:
  name = 'x'+str(i+1)
  if df.iloc[:,i+3].isnull().sum() > 16000:
    dele_list.append(name)
    i+=1
    continue
  value = df.iloc[:,i+3].values.reshape(-1,1)
  imp = imp_mean.fit_transform(value)
  ave = np.mean(df.iloc[:,i+3])
  ave_list.append(ave)
  df.iloc[:,i+3] = imp
  i += 1

print(len(ave_list))
print(len(dele_list))

962
243


In [ ]:
# data drop deleting columns
df.drop(columns=dele_list, inplace=True)
df1.drop(columns=dele_list, inplace=True)

In [ ]:
# imputation for test data set
i = 0
while i < len(ave_list):
  name = 'x'+str(i+1)
  df1.iloc[:, i+3].fillna(ave_list[i], inplace=True)
  i += 1

# Get x, y

In [ ]:
y = df.iloc[:,-1]
x = df.drop(columns=['uniqueid', 'health'])
#x['personid']= x['personid'].map(str)
x_columns = x.columns.values.tolist()
#print(x_columns)
#x

In [ ]:
state = 0
test_size = 0.3  
  
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=state)

# Model Training

## Random Forest

In [ ]:
# tuning

scorel = []
for i in range(0,400,20):
  print(i)
  rfc = RandomForestClassifier(n_estimators=i+1, n_jobs=-1,random_state=123)
  score = cross_val_score(rfc,x, y, cv=5).mean()
  scorel.append(score)


#for max_depth in np.arrange()

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380


In [ ]:
print(scorel)
print(max(scorel),(scorel.index(max(scorel))*20)+1) 

[0.33210629953778503, 0.42211774753611986, 0.4342460344992782, 0.4385563628073862, 0.44539631879949654, 0.4460283044738051, 0.4459132628282007, 0.4477525745606645, 0.4469478612231682, 0.4482698985373654, 0.44930457952972347, 0.4482700306931903, 0.45137356156644304, 0.451776042131277, 0.4520059932666608, 0.4516608352908915, 0.4536150565131346, 0.45269538412742455, 0.453327617593905, 0.4537299329639578]
0.4537299329639578 381


In [ ]:
param_grid = {'max_depth':np.arange(1, 20, 1)}


rfc = RandomForestClassifier(n_estimators=100,random_state=90)

GS = GridSearchCV(rfc,param_grid,cv=10)
GS.fit(x,y)

GridSearchCV(cv=10, estimator=RandomForestClassifier(random_state=90),
             param_grid={'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19])})

In [ ]:
print(GS.best_params_)
print(GS.best_score_)

{'max_depth': 17}
0.4609136906532358


In [ ]:
param_grid={'min_samples_split':np.arange(2, 2+30, 1)}
rfc = RandomForestClassifier(n_estimators=100, random_state=90)

GS = GridSearchCV(rfc,param_grid,cv=10)
GS.fit(x,y)

GridSearchCV(cv=10, estimator=RandomForestClassifier(random_state=90),
             param_grid={'min_samples_split': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])})

In [ ]:
print(GS.best_params_)
print(GS.best_score_)

{'min_samples_split': 26}
0.4651096547758323


In [ ]:
# max feature tuning
param_grid = {'max_features':np.arange(5,205,5)}
rfc = RandomForestClassifier(n_estimators=100, random_state=90)

GS = GridSearchCV(rfc,param_grid,cv=10)
GS.fit(x,y)

In [ ]:
print(GS.best_params_)
print(GS.best_score_)

In [ ]:
# randomforest
n_est = 320
max_d = 13
max_f = 150

clf_rf = RandomForestClassifier(n_estimators=n_est, max_depth=max_d, 
                max_features=max_f, bootstrap=True, 
                oob_score=True, random_state=0)
clf_rf.fit(x_train, y_train)

RandomForestClassifier(max_depth=13, max_features=150, n_estimators=300,
                       oob_score=True, random_state=0)

In [ ]:
# model score

print(clf_rf.oob_score_)
# calculate cross entropy (log-loss) on training data
print(log_loss(y_train, clf_rf.predict_proba(x_train)))

y_pred = clf_rf.predict(x_test)

print(log_loss(y_pred, clf_rf.predict_proba(x_test)))

#print(accuray_score(y_test, y_pred))

0.4769256035473805
0.7265215548028884
0.7836939297887454


In [ ]:
# produce importance
# don't need to run every time
print(clf_rf.n_features_in_)
feature_dic = {}
fea_name = clf_rf.feature_names_in_

fea_importance = clf_rf.feature_importances_

print(fea_name.shape)
print(fea_importance.shape)
for i in range(clf_rf.n_features_in_):
  feature_dic[fea_name[i]] = fea_importance[i]
print(len(feature_dic))
print(feature_dic)

new_dic = sorted(feature_dic.items(), key=lambda x: x[1], reverse=True)
print(new_dic[:20])

In [ ]:
pred = clf_rf.predict_proba(df1.iloc[:,1:]).T
pred = pd.DataFrame(pred)
pred = pred.append(df1['uniqueid'])
pred = pred.T
pred.columns = ['p1', 'p2', 'p3', 'p4', 'p5', 'uniqueid']
pred = pred[['uniqueid', 'p1', 'p2', 'p3', 'p4', 'p5']]
pred['uniqueid'] = pred['uniqueid'].astype(int)
pred.to_csv('result_rf.csv', index=False)

## Logistic Regression

In [ ]:
# Logistic Regression
clf_lr = LogisticRegression(penalty='l1', solver='saga', random_state=0).fit(x_train, y_train)
print(clf_lr.score(x,y))

print(log_loss(y_train, clf_rf.predict_proba(x_train)))

y_pred = clf_rf.predict(x_test)

print(log_loss(y_pred, clf_rf.predict_proba(x_test)))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


0.40464421197838835
0.7265215548028884
0.7836939297887454


In [ ]:
# Logistic Regression Lasso feature selection
print(clf_lr.n_features_in_)
feature_dic = {}
fea_name = clf_lr.feature_names_in_

fea_importance = np.mean(np.abs(clf_lr.coef_), axis=0)

print(fea_name.shape)
print(fea_importance.shape)
for i in range(clf_lr.n_features_in_):
  feature_dic[fea_name[i]] = fea_importance[i]
print(len(feature_dic))
print(feature_dic)

new_dic = sorted(feature_dic.items(), key=lambda x: x[1], reverse=True)
print(new_dic[:20])

In [ ]:
pred_lr = clf_lr.predict_proba(df1.iloc[:,1:]).T
pred_lr = pd.DataFrame(pred_lr)
pred_lr = pred_lr.append(df1_org['uniqueid'])
pred_lr = pred_lr.T
pred_lr.columns = ['p1', 'p2', 'p3', 'p4', 'p5', 'uniqueid']
pred_lr = pred_lr[['uniqueid', 'p1', 'p2', 'p3', 'p4', 'p5']]
pred_lr['uniqueid'] = pred_lr['uniqueid'].astype(int)
pred_lr
pred_lr.to_csv('result_lr.csv', index=False)

## Gradient boosting

### Standardization

In [ ]:
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train = pd.DataFrame(scaler.transform(x_train))
x_train.columns = x_columns
scaler.fit(x_test)
x_test = pd.DataFrame(scaler.transform(x_test))
x_test.columns = x_columns
x_train

df1_columns = df1.columns.values.tolist()
scaler = MinMaxScaler()
scaler.fit(df1.iloc[:, 3:])
df1.iloc[:, 3:] = pd.DataFrame(scaler.transform(df1.iloc[:, 3:]))
df1.columns = df1_columns
df1

In [ ]:
# Gradient Boosting
lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]
estimator_list = [20, 50, 100, 150, 200]
max_features_list = [10, 50, 100, 125, 150, 175]
max_depth_list = [5, 8, 11, 13]
min_samples_split = 10
min_samples_leaf = 5

training_acc = []
testing_acc = []

i = 0

for learning_rate in lr_list:
    for n_estimator in estimator_list:
        for max_feature in max_features_list:
            for max_depth in max_depth_list:
    
                gb_clf = GradientBoostingClassifier(min_samples_split=10, min_samples_leaf=5, n_estimators=n_estimator, learning_rate=learning_rate, max_features=max_feature, max_depth=max_depth, random_state=0)
                gb_clf.fit(x_train, y_train)

                print("Model-{} ".format(i), "Learning rate: ", learning_rate, "n_estimator: ", n_estimator, "max_feature: ", max_feature, "max_depth: ", max_depth)
                print("Accuracy score (training): {0:.3f}".format(gb_clf.score(x, y)))
                print("Accuracy score (validation): {0:.3f}".format(gb_clf.score(x_test, y_test)))
                print('*******************************************************************************')

                training_acc.append(gb_clf.score(x_train, y_train))
                testing_acc.append(gb_clf.score(x_test, y_test))
                i += 1

In [ ]:
print(training_acc)

In [ ]:
gb_clf = GradientBoostingClassifier(n_estimators=300, learning_rate=0.5, max_features=150, max_depth=13, random_state=0)
gb_clf.fit(x_train, y_train)

pred = gb_clf.predict_proba(df1.iloc[:,1:]).T
pred = pd.DataFrame(pred)
pred = pred.append(df1['uniqueid'])
pred = pred.T
pred.columns = ['p1', 'p2', 'p3', 'p4', 'p5', 'uniqueid']
pred = pred[['uniqueid', 'p1', 'p2', 'p3', 'p4', 'p5']]
pred['uniqueid'] = pred['uniqueid'].astype(int)
pred
pred.to_csv('result_gb.csv', index=False)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

seed = 7
np.random.seed(seed)

def model():
  # create model
  model = Sequential()
  model.add(Dense(10, input_dim=964, init='normal', activation='relu'))
  model.add(Dropout(0.1))
  model.add(Dense(5, input_dim=10, init='normal', activation='relu'))
  model.add(Dense(5, input_dim=5, init='normal', activation='softmax'))
  # Compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [ ]:
estimator = KerasClassifier(build_fn=model, nb_epoch=200, batch_size=5, verbose=0)

In [ ]:
kfold = KFold(n=len(x), n_folds=10, shuffle=True, random_state=seed)

In [ ]:
results = cross_val_score(estimator, x, y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))